In [4]:
import pandas as pd

### Overall Iteration variance distribution

In [8]:
def get_eval_table(experiment, col='overall_accuracy'):
    file_name = f'../data/evaluation/{experiment}_eval_table.csv'
    evaluation_df = pd.read_csv(file_name)
    evaluation_df['overall_accuracy'] = evaluation_df['accuracy'].fillna(0)
    evaluation_df['experiment'] = experiment
    return evaluation_df

In [17]:
for col in ['completeness','overall_accuracy', 'structural_compliance']:
    file_names = ['gpt', 'fewshot', 'llama2', 'newgpt','rag']
    eval_dfs = [get_eval_table(file_name, col) for file_name in file_names]
    eval_table_df = pd.concat(eval_dfs)
    evaluation_var_df = eval_table_df.groupby(['sample_id','top_n', 'prompt','gpt_version','input_type','experiment'])[col].sum().reset_index()
    print(evaluation_var_df[col].astype(str).value_counts().reset_index())


  completeness  count
0            3   7837
1            0   1668
2            2    712
3            1    454
  overall_accuracy  count
0              0.0   9433
1              3.0    897
2              1.0    183
3              2.0    158
  structural_compliance  count
0                     0   4898
1                     3   4347
2                     1    714
3                     2    712


#### Breakdown by methods

In [37]:
for col in ['completeness','overall_accuracy', 'structural_compliance']:
    file_names = ['gpt', 'fewshot', 'llama2', 'newgpt','rag']
    eval_dfs = [get_eval_table(file_name, col) for file_name in file_names]
    eval_table_df = pd.concat(eval_dfs)
    evaluation_var_df = eval_table_df.groupby(['sample_id','top_n', 'prompt','gpt_version','input_type','experiment'])[col].sum().reset_index()
    # for col = 1 or 2, we want to change the value to 'diff'
    evaluation_var_df[col].iloc[evaluation_var_df[col].isin([1,2])] = 'diff'
    
    variants_count_df = evaluation_var_df.groupby(['gpt_version','experiment','top_n','prompt','input_type',col]).size().reset_index()
    variants_count_df.rename(columns={0:'col_count'}, inplace=True)
    sum_count_df = evaluation_var_df.groupby(['gpt_version','experiment','top_n','prompt','input_type']).size().reset_index()
    sum_count_df.rename(columns={0:'sum_count'},inplace=True)
    nonzero_count_df = evaluation_var_df[evaluation_var_df[col]!=0].groupby(['gpt_version','experiment','top_n','prompt','input_type']).size().reset_index()
    nonzero_count_df.rename(columns={0:'nonzero_count'},inplace=True)
    merged_df = variants_count_df.merge(sum_count_df).merge(nonzero_count_df)
    merged_df['difference_ratio'] = (merged_df['col_count'])/merged_df['sum_count']
    merged_df['nonzero_difference_ratio'] = (merged_df['col_count'])/merged_df['nonzero_count']
    # for col_count = 2 or 3, we want to add the ratio together
    merged_df = merged_df[merged_df[col]=='diff']
    # drop col
    merged_df.drop(col, axis=1, inplace=True)
    merged_df.sort_values('nonzero_difference_ratio', ascending=False, inplace=True)
    merged_df.to_csv(f'../data/evaluation/{col}_difference_ratio.csv', index=False)

/var/folders/jt/xqkngjbx2vj5dhrgmccfc01m0000gp/T/ipykernel_73521/772243224.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  evaluation_var_df[col].iloc[evaluation_var_df[col].isin([1,2])] = 'diff'
/var/folders/jt/xqkngjbx2vj5dhrgmccfc01m0

#### Investigate Calendar Month Impact

In [61]:
for col in ['completeness','overall_accuracy', 'structural_compliance']:
    file_names = ['gpt', 'newgpt']
    eval_dfs = [get_eval_table(file_name, col) for file_name in file_names]
    eval_table_df = pd.concat(eval_dfs)
    # pivot table from long to wide by experiment
    pivot_df = eval_table_df.pivot_table(index=['sample_id','top_n','prompt','gpt_version','input_type','iteration'], columns='experiment', values=col).reset_index()
    pivot_df = pivot_df[~pivot_df['newgpt'].isna()]
    pivot_df = pivot_df[~pivot_df['gpt'].isna()]
    gpt_evaluation_var_df = pivot_df.groupby(['sample_id','top_n', 'prompt','gpt_version','input_type'])['gpt'].sum().reset_index()
    newgpt_evaluation_var_df = pivot_df.groupby(['sample_id','top_n', 'prompt','gpt_version','input_type'])['newgpt'].sum().reset_index()
    merged_df = gpt_evaluation_var_df.merge(newgpt_evaluation_var_df)
    merged_df[f'{col}_abs_diff'] = abs(merged_df['gpt'] - merged_df['newgpt'])
    print(merged_df.groupby([f'{col}_abs_diff']).size().reset_index())

   completeness_abs_diff    0
0                    0.0  211
1                    1.0   11
2                    2.0    7
3                    3.0   21
   overall_accuracy_abs_diff    0
0                        0.0  240
1                        1.0    2
2                        2.0    3
3                        3.0    5
   structural_compliance_abs_diff    0
0                             0.0  125
1                             1.0    5
2                             2.0    2
3                             3.0  118


(completeness_variability_df[0] == 2).sum()

,abs_diff,0
0,0.0,125
1,1.0,5
2,2.0,2
3,3.0,118


In [59]:
pivot_df

experiment,sample_id,top_n,prompt,gpt_version,input_type,iteration,gpt,newgpt
12594,Init_example_1.0,10,d,gpt-3.5-turbo,free_text,1,0.0,1.0
12595,Init_example_1.0,10,d,gpt-3.5-turbo,free_text,2,0.0,1.0
12596,Init_example_1.0,10,d,gpt-3.5-turbo,free_text,3,0.0,1.0
12597,Init_example_1.0,10,d,gpt-4,free_text,1,1.0,1.0
12598,Init_example_1.0,10,d,gpt-4,free_text,2,1.0,1.0
...,...,...,...,...,...,...,...,...
18547,PMID30709875.4,10,d,gpt-3.5-turbo,free_text,2,0.0,1.0
18548,PMID30709875.4,10,d,gpt-3.5-turbo,free_text,3,0.0,1.0
18549,PMID30709875.4,10,d,gpt-4,free_text,1,1.0,1.0
18550,PMID30709875.4,10,d,gpt-4,free_text,2,1.0,1.0
